In [31]:
'''
Реализуйте функцию с 4 роторами:

0 - "не шифрованный ротор" - возвращает всегда тот же символ, что и принимает (в любом направлении)
1-3 - роторы номер 1, 2 и 3 соответственно.
Спецификация на роторы (и таблица соответствия символов) - http://www.codesandciphers.org.uk/enigma/rotorspec.htm

Реализуйте функцию rotor(symbol, n, reverse=False)

symbol - символ, поступающий для шифрования
n - номер ротора
reverse - признак обратного направления, если находится в значении True, это значит, что функцию надо использовать в обратном направлении. (по умолчанию False)
Возвращает строку с зашифрованным символом
'''

rotors = {
    0: 'ABCDEFGHIJKLMNOPQRSTUVWXYZ',
    1: 'EKMFLGDQVZNTOWYHXUSPAIBRCJ',
    2: 'AJDKSIRUXBLHWTMCQGZNPYFVOE',
    3: 'BDFHJLCPRTXVZNYEIWGAKMUSQO'
}

L = len(rotors[0])

def rotor(symbol, n, reverse=False):
    if symbol.isalpha():
        rot_from, rot_to = (rotors[0], rotors[n]) if not reverse else (rotors[n], rotors[0])
        return rot_to[rot_from.index(symbol)]

rotor('A', 1)

'E'

In [19]:
'''
Реализуйте функцию "отражатель". Пока для нас она аналогична ротору, за 2 исключениями:

Имеет только 1 направление
Отражатель с функцией шифрования всегда соединяет символы парами
Реализуйте 2 отражателя:

0 - без шифрования
1 - отражатель вида "B"
Спецификация на Отражатели (и таблица соответствия символов) - http://www.codesandciphers.org.uk/enigma/rotorspec.htm

 

Реализуйте функцию reflector(symbol, n):

symbol - символ, поступающий для шифрования
n - номер отражателя
'''

reflectors = {
    0: 'ABCDEFGHIJKLMNOPQRSTUVWXYZ',
    1: 'YRUHQSLDPXNGOKMIEBFZCWVJAT'
}

def reflector(symbol, n):
    return reflectors[n][reflectors[0].index(symbol)] if symbol.isalpha() else ''

reflector('A', 1)

'Y'

In [20]:
'''
Смещения и движение роторов/отражателей отсутствуют. Фактически мы реализуем примеры 1 и 2 с предыдущего шага.
Реализуйте функцию enigma(text, ref, rot1, rot2, rot3), где:

text - исходный текст, который необходимо зашифровать
ref - номер отражателя (согласно задаче https://stepik.org/lesson/283487/step/3)
rot1, rot2, rot3 - номера роторов (согласно задаче https://stepik.org/lesson/283487/step/2)
Функция возвращает зашифрованный текст.

Сигнал распространяется по машине справа налево (и после отражения обратно)
'''

def enigma(text, ref, rot1, rot2, rot3):
    res = ''
    for letter in text.upper():
        if not letter.isalpha():
            continue
        s = rotor(letter, rot3)
        s = rotor(s, rot2)
        s = rotor(s, rot1)
        s = reflector(s, ref)
        s = rotor(s, rot1, reverse=True)
        s = rotor(s, rot2, reverse=True)
        res += rotor(s, rot3, reverse=True)
    return res

enigma('Some encripted text', 1, 1, 2, 3)

'LDRBBKJMWGFBOFBYF'

In [42]:
'''Реализуйте функцию enigma(text, ref, rot1, shift1, rot2, shift2, rot3, shift3) с поворачивающимися роторами, как они описаны на предыдущем шаге.

text - исходный текст, который необходимо зашифровать
ref - номер отражателя (согласно задаче https://stepik.org/lesson/283487/step/3)
rot1, rot2, rot3 - номера роторов (согласно задаче https://stepik.org/lesson/283487/step/2)
shift1, shift2, shift3 - сдвиги роторов (1, 2 и 3, соответственно)
'''

def shift(letter, step):
    return rotors[0][(rotors[0].index(letter) + step) % L]

def enigma(text, ref, rot1, shift1, rot2, shift2, rot3, shift3):
    res = ''
    for letter in text.upper():
        if not letter.isalpha():
            continue
        s = rotor(shift(letter, shift3), rot3)
        s = rotor(shift(s, shift2-shift3), rot2)
        s = rotor(shift(s, shift1-shift2), rot1)
        s = reflector(shift(s, -shift1), ref)
        s = rotor(shift(s, shift1), rot1, reverse=True)
        s = rotor(shift(s, shift2-shift1), rot2, reverse=True)
        s = rotor(shift(s, shift3-shift2), rot3, reverse=True)
        res += shift(s, -shift3)
    return res

enigma('A', 1, 2, 4, 2, 5, 2, 6)

'D'

In [44]:
'''
Реализуйте функцию enigma(text, ref, rot1, shift1, rot2, shift2, rot3, shift3) с поворачивающимися и двигающимися роторами
'''
shifts = {
    1: 17-1,
    2: 5-1,
    3: 22-1
}

def new_shift(rot1, shift1, rot2, shift2, rot3, shift3):
    if shift2 == shifts[rot2]:
        shift1 = (shift1 + 1) % L
        shift2 = (shift2 + 1) % L
    if shift3 == shifts[rot3]:
        shift2 = (shift2 + 1) % L
    shift3 = (shift3 + 1) % L
    return (shift1, shift2, shift3)

new_shift(2, 3, 2, 4, 2, 5)

def enigma(text, ref, rot1, shift1, rot2, shift2, rot3, shift3):
    res = ''
    for letter in text.upper():
        if not letter.isalpha():
            continue
        shift1, shift2, shift3 = new_shift(rot1, shift1, rot2, shift2, rot3, shift3)
        s = rotor(shift(letter, shift3), rot3)
        s = rotor(shift(s, shift2-shift3), rot2)
        s = rotor(shift(s, shift1-shift2), rot1)
        s = reflector(shift(s, -shift1), ref)
        s = rotor(shift(s, shift1), rot1, reverse=True)
        s = rotor(shift(s, shift2-shift1), rot2, reverse=True)
        s = rotor(shift(s, shift3-shift2), rot3, reverse=True)
        res += shift(s, -shift3)
    return res

enigma('AAAAAAA', 1, 2, 3, 2, 3, 2, 3)

'BGDMBTZ'